In [36]:
try:
    import sys
    import random
    import math
    from datetime import datetime
    from flask import Flask, request
    from flask_restful import Resource, Api
    from flask_cors import CORS
    
    datetimeNowBegin = datetime.now()
    sys.stdout.write("Début du script: " + str(datetimeNowBegin))
except:
    %run Install.ipynb

Début du script: 2019-04-07 22:57:16.300317

In [37]:
#Global variables
MIN = 0 
MAX = 1024
FIRST_RADIUS = 100
NB_PLAYERS = 10

In [38]:
#Global functions
def randomEvent(proba):
        """
        Simulate a random decision among "go back" (negative direction), "stay" (idle) and "go straight on" 
        (positive direction) on an axis.
        Args:
            proba: Array of the 3 probabilities [negative direction, idle, positive direction]
        Return:
            int: the direction of the movement. -1 is negative direction, 0 idle, 1 positive direction
        """
        proba = proba[:]
        proba[1] += proba[0]
        proba[2] += proba[1]
        
        if proba[2] > 1:
            print("Error: Proba > 1", proba)
            return None
        else:
            rand = random.random()
            index = 0
            while proba[index] < rand:
                index += 1
            return index - 1

        
def tirage_disque(x0, R, N=1):
    """ http://www.afapl.asso.fr/Tiralea.htm """
    res = list()
    for _ in range(N):
        u = random.random()
        radius = R * math.sqrt(u)
        theta = random.uniform(0., 2 * math.pi)
        x = x0[0] + radius * math.cos(theta)
        y = x0[1] + radius * math.sin(theta)
        res.append((int(x), int(y)))
    if N == 1:
        return res[0]
    return res

In [39]:
class Player():
    """
    Player Class.
    Attr:
        _positions: Array of position tuple (x, y), with x and y integers in [MIN, MAX].
        _color: Tuple (r, g, b), with r, g and b integers between 0 and 255. Used in display.
    """

    def __init__(self, start=None):
        """
        Create a player.
        Args:
            start: (x, y) the starting point of the player. When no "start" given, the first position is 
            a random point between MIN and MAX.
        Return: 
            None. It creates a new Player.
        """
        self._positions= []
        if start:
            self._positions.append(start)
        else:
            self._positions.append((int(random.random() * (MAX - MIN) + MIN), int(random.random() * (MAX - MIN) + MIN)))
            
        self._color= (int(random.random()*256), int(random.random()*256), int(random.random()*256))
    
    
    def move(self, zone, players):
        """
        Define how the player moves.
        This function can be edited to change player behaviours.
        Args: 
            zone: A Zone object. Can be used when movement depends on the zone.
            players: Array of Player objects. Can be used when movement depends on the other players.
        Return: 
            None. It just adds a position tuple (x, y) to the positions Array of the Player.
        """
        probaX = [1/3, 1/3, 1/3]
        probaY = [1/3, 1/3, 1/3]
        
        moveX = randomEvent(probaX)
        moveY = randomEvent(probaY)
        
        #To have borders
        while self._positions[-1][0] + moveX > MAX or self._positions[-1][0] + moveX < MIN:
            moveX = randomEvent(probaX)
        while self._positions[-1][1] + moveY > MAX or self._positions[-1][1] + moveY < MIN:
            moveY = randomEvent(probaY)
        
        self._positions.append((self._positions[-1][0] + moveX, self._positions[-1][1] + moveY))
    
    
    def get_positions(self):
        """
        Getter of Player positions.
        Return: 
            Array of position tuple (x, y).
        """
        return self._positions
                                   
    
    def get_color(self):
        """
        Getter of Player color.
        Return: 
            Tuple (red, green, blue)
        """
        return self._color
    
    
class Zone():
    """
    Zone Class.
    Attr:
        _number: Integer between 1 and 9.
        _center: Position tuple (x, y), with x and y integers in [MAX/4, 3*MAX/4].
    """
    
    #Class variables:
    #Time - Zone-number mapping.
    timer = {60: 1, 260: 1, 440: 2, 560: 2, 680: 3, 770: 3, 860: 4, 940: 4, 1010: 5, 1060: 5, 1120: 6, 1150: 6, 1210: 7, 1265: 8, 1310: 9}
    #Zone radius for each zone number.
    radius = [None, FIRST_RADIUS*4, FIRST_RADIUS*2, FIRST_RADIUS, FIRST_RADIUS/2, FIRST_RADIUS/4, FIRST_RADIUS/8, FIRST_RADIUS/16, FIRST_RADIUS/32, FIRST_RADIUS/64]
    
    def __init__(self, start=None):
        """
        Create a zone.
        Args:
            start: (x, y) the starting point of the zone. When no "start" given, the first position is 
            a random point between MAX/4 and 3*MAX/4.
        Return: 
            None. It creates a new Zone.
        """
        self._number = 1
        
        if start:
            self._center = start
        else:
            self._center = (int(random.random() * MAX/2 + MAX/4), int(random.random() * MAX/2 + MAX/4))
            
            
    def next_zone(self, time):
        """
        Link the time to the zone attributes.
        Args:
            time: Integer.
        Return: 
            None. It changes the zone attribute if needed.
        """
        zone_index = 0
        while zone_index < len(Zone.timer.keys()) and int(list(Zone.timer.keys())[zone_index]) < int(time):
            zone_index += 1
            
        if zone_index < len(Zone.timer.keys()):
            if self._number != Zone.timer[int(list(Zone.timer.keys())[zone_index])]:
                
                self._number = Zone.timer[int(list(Zone.timer.keys())[zone_index])]
                self._center = tirage_disque(self._center, Zone.radius[self._number - 1] - Zone.radius[self._number])
            
            
    def get_center(self):
        """
        Getter of Zone center.
        Return: 
            Position tuple (x, y)
        """
        return self._center

    
    def get_radius(self):
        """
        Getter of Zone radius.
        Return: 
            Integer
        """
        return Zone.radius[self._number]
    
    
    def get_number(self):
        """
        Getter of Zone number.
        Return: 
            Integer
        """
        return self._number

In [ ]:
# ----- Model of the random walk ----- #

#Creation of players
players = [Player() for i in range(NB_PLAYERS)]
#Creation of the zone
zone = Zone()
#Timer
timer = 0


In [ ]:
#API Requests. 
#These functions should not be changed.

class PlayersPositionsRequest(Resource):
    
    def get(self):
        """
        GET request that return the positions of every players.
        Return:
            array of positions
        """
        
        data = [{"positions": player.get_positions(), "color": player.get_color()} for player in players]
        
        return data

    
class MovePlayersRequest(Resource):
    
    def get(self, frames):
        global timer
        """
        GET request that moves players "frames" times
        Return:
            timer: integer
        """
        for i in range(int(frames)):
            for player in players:
                player.move(zone, players)
        
        timer += 100
        
        zone.next_zone(timer)
                
        return {'timer': timer, 'zone': zone.get_number()}
    

class ResetRequest(Resource):
    
    def get(self):
        global players
        """
        GET request that reset the position of the players
        """
        
        
        return {}
    

class ZonePositionRequest(Resource):
    
    def get(self):
        """
        GET request that return Zone data
        Return:
            center: Position tuple (x, y) and radius: integer
        """
        return {'center': zone.get_center(), 'radius': zone.get_radius()}
    

class ResetRequest(Resource):
    
    def get(self):
        global players, zone, timer
        """
        GET request that reset the model
        """
        players = [Player() for i in range(NB_PLAYERS)]
        zone = Zone()
        timer = 0
        
        return {}

In [ ]:
app = Flask(__name__)
api = Api(app)

CORS(app)

api.add_resource(PlayersPositionsRequest, '/playerspositions')
api.add_resource(MovePlayersRequest, '/moveplayers/<frames>')
api.add_resource(ZonePositionRequest, '/zoneposition')
api.add_resource(ResetRequest, '/reset')

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Apr/2019 22:57:19] "GET /reset HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:20] "GET /moveplayers/100 HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:20] "GET /playerspositions HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:20] "GET /zoneposition HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:20] "GET /moveplayers/100 HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:20] "GET /playerspositions HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:21] "GET /playerspositions HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:21] "GET /moveplayers/100 HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:21] "GET /zoneposition HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:21] "GET /moveplayers/100 HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:21] "GET /playerspositions HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:22] "GET /moveplayers/100 HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2019 22:57:22] "GET /playerspo